# Pytorchを使ってSeq2Seqを実装する

In [8]:
from pathlib import Path
import time
import math
import pandas as pd
import numpy as np
from torch import optim
import random
import requests
import zipfile
import urllib
import codecs
import re
import neologdn
import unicodedata
import MeCab
from collections import Counter
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.vocab import Vocab
from load_data import *

## モデルの設定

In [9]:
class Encoder(nn.Module):
    
    def __init__(
        self, vocab_size_src, embedding_dim, hidden_dim_enc, dropout, PAD_IDX
    ):
        
        super().__init__()

        self.vocab_size_src = vocab_size_src
        
        self.embedding = nn.Embedding(vocab_size_src, embedding_dim, padding_idx=PAD_IDX)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim_enc)

    def forward(self, src):

        embedded = self.dropout(self.embedding(src))
        _, state = self.rnn(embedded)
        
        return state

In [10]:
class Decoder(nn.Module):
    
    def __init__(
        self, vocab_size_trg, embedding_dim, hidden_dim_dec, dropout, PAD_IDX
    ):
        
        super().__init__()

        self.vocab_size_trg = vocab_size_trg

        self.embedding = nn.Embedding(vocab_size_trg, embedding_dim, padding_idx=PAD_IDX)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim_dec)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_dim_dec, vocab_size_trg)

    def forward(self, x, encoder_state):

        x = x.unsqueeze(0)
        embedded = self.dropout(self.embedding(x))
        output, state = self.rnn(embedded, encoder_state)
        output = self.out(output[0])

        return output, state

In [11]:
class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder):
        
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        batch_size = src.shape[1]
        seq_len = trg.shape[0]
        vocab_size_trg = self.decoder.vocab_size_trg
        
        state = self.encoder(src)
        
        outputs = torch.zeros(seq_len, batch_size, vocab_size_trg)
        output = trg[0, :]
        for t in range(1, seq_len):
            output, state = self.decoder(output, state)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            output = trg[t] if teacher_force else output.max(1)[1]

        return outputs

In [12]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.02)
        else:
            nn.init.constant_(param.data, 0)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [13]:
def train(model, data, optimizer, criterion, clip):

    model.train()

    losses = []
    for src, trg in tqdm(data):

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        losses.append(loss.item())

    return np.mean(losses)


def evaluate(model, data, criterion):

    model.eval()

    losses = []
    with torch.no_grad():

        for src, trg in data:

            output = model(src, trg, 0)

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            losses.append(loss.item())

    return np.mean(losses)

## SeqSeq2の実行

In [15]:
embedding_dim_enc = 32
embedding_dim_dec = 32
hidden_dim_enc = 64
hidden_dim_dec = 64
dropout_enc = 0.5
dropout_dec = 0.5
vocab_size_src = len(de_vocab)
vocab_size_trg = len(en_vocab)

encoder = Encoder(
    vocab_size_src=vocab_size_src, 
    embedding_dim=embedding_dim_enc, 
    hidden_dim_enc=hidden_dim_enc,
    dropout=dropout_enc,
    PAD_IDX=PAD_IDX
)

decoder = Decoder(
    vocab_size_trg=vocab_size_trg, 
    embedding_dim=embedding_dim_dec,
    hidden_dim_dec=hidden_dim_dec, 
    dropout=dropout_dec,
    PAD_IDX=PAD_IDX
)

model = Seq2Seq(encoder=encoder, decoder=decoder)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters())

In [16]:
epochs = 100
clip = 1
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
patience = 10
best_loss = float('inf')
best_model = None

losses_train = []
losses_valid = []
counter = 0
for epoch in range(epochs):

    start_time = time.time()

    loss_train = train(
        model=model, 
        data=train_iter, 
        optimizer=optimizer, 
        criterion=criterion, 
        clip=clip
    )
    
    elapsed_time = time.time() - start_time
    
    loss_valid = evaluate(
        model=model, 
        data=valid_iter, 
        criterion=criterion
    )
    
    print('[{}/{}] train loss: {:.2f}, valid loss: {:.2f}  [{}{:.0f}s] {}'.format(
        epoch + 1, epochs,
        loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        '**' if best_loss > loss_valid else ''
    ))
    
    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = model
        counter = 0
        
    if counter > patience:
        break
        
    counter += 1

  0%|          | 0/227 [00:00<?, ?it/s]

[1/100] train loss: 6.04, valid loss: 5.35  15s] **


  0%|          | 0/227 [00:00<?, ?it/s]

[2/100] train loss: 5.24, valid loss: 5.22  8s] **


  0%|          | 0/227 [00:00<?, ?it/s]

...

[71/100] train loss: 3.24, valid loss: 4.36  56s] 


  0%|          | 0/227 [00:00<?, ?it/s]

[72/100] train loss: 3.23, valid loss: 4.35  50s] 


100%|██████████| 227/227 [08:53<00:00,  2.35s/it]


[73/100] train loss: 3.22, valid loss: 4.38  53s] 


In [33]:
def translate(
    model, text, vocab_src, vocab_trg, tokenizer_src, max_seq_length_trg
):
    
    tokens = [BOS_IDX] + [vocab_src.stoi[tok] for tok in tokenizer_src(text)] + [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    
    model.eval()
    predict_words = greedy_decode(
        model=model, src=src, vocab_trg=vocab_trg,
        max_seq_length_trg=max_seq_length_trg
    )
    return ' '.join(predict_words)


def greedy_decode(model, src, vocab_trg, max_seq_length_trg):
    
    state = model.encoder(src)

    predict_words = []
    output = torch.tensor([BOS_IDX])
    for t in range(1, max_seq_length_trg):
        output, state = model.decoder(output, state)
        output = output.max(1)[1]
        if output == EOS_IDX:
            break
        predict_words.append(vocab_trg.itos[output])

    return predict_words

In [34]:
text = "Eine Gruppe von Menschen steht vor einem Iglu ."
translate(
    model=best_model, text=text, vocab_src=de_vocab,
    vocab_trg=en_vocab, tokenizer_src=de_tokenizer,
    max_seq_length_trg=100
)

'A crowd of a a a . .'

In [37]:
model=best_model
text=text
vocab_src=de_vocab
vocab_trg=en_vocab
tokenizer_src=de_tokenizer

In [38]:
tokens = [BOS_IDX] + [vocab_src.stoi[tok] for tok in tokenizer_src(text)] + [EOS_IDX]
num_tokens = len(tokens)
src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )

In [40]:
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19215, 32, padding_idx=1)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(32, 64)
  )
  (decoder): Decoder(
    (embedding): Embedding(10838, 32, padding_idx=1)
    (rnn): LSTM(32, 64)
    (dropout): Dropout(p=0.5, inplace=False)
    (out): Linear(in_features=64, out_features=10838, bias=True)
  )
)

In [41]:
state = model.encoder(src)

In [43]:
output = torch.tensor([BOS_IDX])

In [45]:
output, state = model.decoder(output, state)

In [49]:
output = output.max(1)[1]

In [51]:
output, state = model.decoder(output, state)

In [56]:
output.max(1)[1]

tensor([94])

In [57]:
en_vocab.itos[94]

'crowd'